<a href="https://colab.research.google.com/github/walkeradonis/landing_example/blob/master/Proyectoparade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas

Ejercicio 1:

Utiliza Pandas y el conjunto de datos público de COVID-19 proporcionado por la
Universidad de Johns Hopkins para realizar las siguientes tareas:
a) Descarga los datos de COVID-19 en formato CSV o JSON desde la URL
pública.
b) Carga los datos en un DataFrame de Pandas.
c) Calcula el promedio de casos confirmados por día en un país específico.
d) Encuentra los 10 países con la tasa de mortalidad más alta (número de
muertes / número de casos confirmados) hasta la fecha.
Puedes encontrar el conjunto de datos de COVID-19 en la URL pública, como la
ofrecida por la Universidad de Johns Hopkins:
https://github.com/CSSEGISandData/COVID-19


In [2]:
import pandas as pd
import requests
from io import StringIO

# inciso a: Descargar los datos de casos confirmados, muertes y recuperados.
base_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/"
confirmed_url = base_url + "csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
deaths_url = base_url + "csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"

# inciso b: Descargar datos en dataframes
confirmed_df = pd.read_csv(confirmed_url)
deaths_df = pd.read_csv(deaths_url)

# inciso c: Promedio de casos confirmados por día para un país específico, en este ejemplo, use a Guatemala
def avg_daily_cases(country):
    country_data = confirmed_df[confirmed_df['Country/Region'] == country]
    daily_cases = country_data.iloc[:, 4:].sum()
    daily_new_cases = daily_cases.diff().fillna(0)
    average = daily_new_cases.mean()
    return average

print("Promedio de casos diarios en Guatemala:", avg_daily_cases("Guatemala"))

# Agrupar por pais
confirmed_latest = confirmed_df.groupby('Country/Region').sum().iloc[:, -1]
deaths_latest = deaths_df.groupby('Country/Region').sum().iloc[:, -1]

# Calculo de la tasa de mortalidad
mortality_rate = (deaths_latest / confirmed_latest).fillna(0).replace([float('inf'), -float('inf')], 0)
top_10_mortality_countries = mortality_rate.sort_values(ascending=False).head(10)

# inciso d: impresion de tasa de mortalidad
print("\nLos 10 países con mayor tasa de mortalidad:")
print(top_10_mortality_countries)


Promedio de casos diarios en Guatemala: 1083.3307086614172

Los 10 países con mayor tasa de mortalidad:
Country/Region
Korea, North              6.000000
MS Zaandam                0.222222
Yemen                     0.180745
Sudan                     0.078601
Syria                     0.055058
Somalia                   0.049810
Peru                      0.048922
Egypt                     0.048108
Mexico                    0.044523
Bosnia and Herzegovina    0.040525
Name: 3/9/23, dtype: float64


In [3]:
pip install pyspark

In [5]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg

# Dataset en Pandas
datos_peliculas = pd.DataFrame({
    'ID': [1, 2, 3, 4],
    'Título': ['Película1', 'Película2', 'Película3', 'Película4'],
    'Año': [2020, 2019, 2021, 2018]
})

# Iniciar sesion de Spark
spark = SparkSession.builder.appName("IntegracionPandasSpark").getOrCreate()

# Dataset en Spark
criticas = spark.createDataFrame([
    (1, "Critico1", 4.5),
    (2, "Critico2", 3.8),
    (3, "Critico1", 4.2),
    (4, "Critico3", 4.7)
], ["PeliculaID", "Critico", "Puntuacion"])

# Agrupar por ID de pelicula, calcular promedio
promedios_spark = criticas.groupBy("PeliculaID").agg(avg("Puntuacion").alias("PuntuacionPromedio"))

promedios_pandas = promedios_spark.toPandas()

# Realizar Join:  datos_peliculas.ID == promedios_pandas.PeliculaID
resultado = datos_peliculas.merge(promedios_pandas, left_on="ID", right_on="PeliculaID")

# Seleccion de columnas
resultado_final = resultado[["Título", "Año", "PuntuacionPromedio"]]

# Imprimir resultado
print(resultado_final)


      Título   Año  PuntuacionPromedio
0  Película1  2020                 4.5
1  Película2  2019                 3.8
2  Película3  2021                 4.2
3  Película4  2018                 4.7


Ejercicio 4: Web Scrapping

Supongamos que deseas obtener el precio actual del Bitcoin (BTC) de un sitio
web de criptomonedas. La información que necesitas se encuentra en la página
de https://coinmarketcap.com/currencies/bitcoin/.
Tu tarea es escribir un script en Python que realice lo siguiente:
1. Realiza una solicitud GET a la URL
https://coinmarketcap.com/currencies/bitcoin/ utilizando la biblioteca
`requests`.
2. Analiza el contenido de la página web para extraer el precio actual del
Bitcoin.
3. Imprime el precio en la consola.
A continuación, un esqueleto del código para empezar:
```python
import requests
from bs4 import BeautifulSoup
# URL de la página de CoinMarketCap
url = 'https://coinmarketcap.com/currencies/bitcoin/'
# Realiza una solicitud GET para obtener el contenido de la página
response = requests.get(url)
# Verifica si la solicitud fue exitosa (código de respuesta 200)
if response.status_code == 200:
 # Analiza el contenido de la página web con BeautifulSoup
 soup = BeautifulSoup(response.text, 'html.parser')
 # Encuentra el elemento que contiene el precio actual del Bitcoin
 # Pista: Inspecciona la página web para identificar el elemento
adecuado
 # Extrae el precio del elemento y almacénalo en una variable
 # Imprime el precio en la consola
else:
 print(f'Error al hacer la solicitud. Código de respuesta:
{response.status_code}')
```
La tarea es completar el código, identificar el elemento HTML que contiene el
precio del Bitcoin y extraer ese valor. Luego, imprime el precio en la consola.


In [9]:
import requests
from bs4 import BeautifulSoup

# URL de CoinMarketCap
url = 'https://coinmarketcap.com/currencies/bitcoin/'

# Encabezados para simulacion de navegador, mozilla por estabilidad
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    # Analizando el contenido con BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Busca el elemento que contiene el precio, el dia de hoy 2025-06-22 es esa clase esyGGG
    price_tag = soup.find('span', class_='esyGGG')

    if price_tag:
        price = price_tag.text
        print(f'El precio actual de Bitcoin en CoinMarketCap es: {price}')
    else:
        print('No se pudo encontrar el precio en la página CoinMarketCap.')
else:
    print(f'Error al hacer la solicitud. Código de respuesta: {response.status_code}')

El precio actual de Bitcoin en CoinMarketCap es: $101,119.04


Serie de preguntas:

Respuestas estan en negrita.

¿Cuál de las siguientes plataformas de Microsoft es una solución de análisis de big data en
la nube?
a) Azure SQL Database
b) **Azure Synapse Analytics**
c) Azure Data Factory
d) Azure Active Directory

En el contexto de Azure Data Factory, ¿cuál de las siguientes actividades se utiliza para
transformar y limpiar datos en un flujo de trabajo?
a) HDInsight Spark
b) Azure Databricks
c) **Data Flow**
d) Azure Stream Analytics

¿Cuál de las siguientes opciones es una característica clave de Apache Spark que permite
procesar datos en memoria para un rendimiento más rápido?
a) Apache Hadoop
b) Apache Flink
c) Spark Streaming
d) **Resilient Distributed Dataset (RDD)**

En el contexto de Pandas, ¿cuál de las siguientes operaciones se utiliza para eliminar filas
duplicadas de un DataFrame?
a) df.groupby()
b) **df.drop_duplicates()**
c) df.fillna()
d) df.pivot_table()

¿Qué lenguaje de programación se utiliza comúnmente en Azure Databricks para el
procesamiento de datos y análisis?
a) R
b) Java
c) **Scala**
d) C#
